In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import statistics
from math import pi
from matplotlib.gridspec import GridSpec
from dateutil.relativedelta import *
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns
import scipy.stats
import pingouin as pg
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [136]:
# COMMON VARIABLES

alg_to_analyze = [
    'vader_sentiment_analysis_score',
    'vader_sentiment_analysis_polarity'
]

tpd_columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

In [137]:
!python3 '/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/main_generate_json_files.py'

Starting script...
Removing existing file...
Ending script...


In [138]:
data = pd.read_json(r'/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/alldata.json')
data.head()

,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43
id_str_twitter,23781146,15148309,927028057763995648,77648005,3429765093,10794662,1034514325644431361,529445600,12050562,62314491,18289937,172880784,18609547,2164986296,1026783544457273345,109604376,1040939562774790144,69734383
name,Fe Rebelatto,Sommelier de Airbnb,✪ Felippe,Conference Distinguished Specialist,Paula Santana,Maharaja of the Legacies,Jessica,"var nickname = ""Riquettinha"";",Roberta Arcoverde,Gabriela,Thamara Andrade,felipe.ts ⚡,Juan Lopes,Rafael Dantas,iagor com i,T̷o̷m̷o̷y̷o̷ ̷ F̷u̷r̷a̷c̷ã̷o̷,Esdras Xavier,belle-🏴.dev || twitch.tv/bellesamways
screen_name,rebelatto,Guilh_rm_,FelippeRegazio,NannoKa,psanrosa13,rponte,dev_jessi,Riquettinha,rla4,Gabrielathalita,thamyk,felipefialho_,juanplopes,RaffaelDantass,Iagor51,RafaelMansilha,esdras_xavieer,bellesamways
location,"Rio de Janeiro, Brasil",somewhere in Brazil,,Ctba-PR Brasil,Brasil,"Ceará, Brazil",São Paulo - Brasil,"Belo Horizonte, Brasil","Sao Paulo, Brazil",Belo Horizonte,"Belo Horizonte, Brazil.",Dark Side of the Moon,"Rio de Janeiro, Brasil","São Paulo, Brasil",Pvh,"Palmas, Tocantins, Brasil","Jaraguá do Sul, Brasil",Curitiba
url,None,https://t.co/gX4oRZWHg3,https://t.co/KkaPlVH4Wu,https://t.co/hDni4Fh2Ng,https://t.co/YJQuDtv2YW,https://t.co/3yZ94QNQvs,https://t.co/MtUeSEBICu,https://t.co/QU71LJgSuD,None,https://t.co/AmkOKWlftK,https://t.co/JWBzdmQOcV,https://t.co/yPuEuh1bn9,https://t.co/Uhw9rEUK2v,None,None,https://t.co/9cS34v6hKc,https://t.co/H7Mzn0CEHR,https://t.co/bf6AeBboQk


In [139]:
def clean_df(df):
    for c in clean_df_columns:
        del df[c]

In [140]:
def create_tweets_pd(data, id_user):

    tweets_pd = pd.DataFrame(data[id_user]['tweets'])
    del tweets_pd[0]
    del tweets_pd[1]
    del tweets_pd[4]
    del tweets_pd[5]
    del tweets_pd[6]
    del tweets_pd[18]
    del tweets_pd[19]
    del tweets_pd[20]
    
    tweets_pd.columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]
 
    return tweets_pd

In [141]:
# VADER X OPLEXICON

clean_df_columns = [
#     'text',
    'created_at',
    'id_user',
    'final_score',
    'final_polarity',
#     'oplexicon_sentiment_analysis_score',
#     'oplexicon_sentiment_analysis_polarity',
    'sentistrength_sentiment_analysis_score',
    'sentistrength_sentiment_analysis_polarity',
    'sentilexpt_sentiment_analysis_score',
    'sentilexpt_sentiment_analysis_polarity'
]

tweets = [data[d]['tweets'] for d in data]

final_tweets = []
for i in tweets:
    final_tweets.extend(i)
    
df = pd.DataFrame(final_tweets)

del df[0]
del df[1]
del df[4]
del df[5]
del df[6]
del df[18]
del df[19]
del df[20]

df.columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

clean_df(df)

df['vader_prediction'] = df['vader_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
df['oplexicon_prediction'] = df['oplexicon_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))

df['accuracy'] = df.apply(lambda x: 1 if x['vader_prediction'] == x['oplexicon_prediction'] else 0, axis=1)

df.head()

,text,vader_sentiment_analysis_score,vader_sentiment_analysis_polarity,oplexicon_sentiment_analysis_score,oplexicon_sentiment_analysis_polarity,vader_prediction,oplexicon_prediction,accuracy
0,"Atenção, estou mudando os moveis da sala de lu...",-0.4588,neg,0.000000,neu,-1,0,0
1,RT @andreariell: o cancelamento dos participan...,0.0000,neu,0.000000,neu,0,0,1
2,RT @AndrewBrobston: NOVA ANTECIPAÇÃO\nBoa tard...,0.7734,pos,0.458831,pos,1,1,1
3,"@DouglasMarendaz Eu amo muito meu pai, mas fal...",-0.8720,neg,0.458831,pos,-1,1,0
4,@lilicebk Eu cresci numa casa cheia de gente e...,-0.6808,neg,0.000000,neu,-1,0,0


In [142]:
def conf_matrix(x, col1, col2):
    if x[col1] == 1 and x[col2] == 1:
        return 'TP'
    elif x[col1] == 1 and x[col2] == -1:
        return 'FN'
    elif x[col1] == -1 and x[col2] == 1:
        return 'FP'
    elif x[col1] == -1 and x[col2] == -1:
        return 'TN'
    else:
        return 0

In [143]:
df['conf_matrix'] = df.apply(lambda x: conf_matrix(x, 'vader_prediction', 'oplexicon_prediction'), axis=1)

conf_vals = df['conf_matrix'].value_counts().to_dict()
print(conf_vals)

accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
precision = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])
recall = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
f1_score = 2*precision*recall / (precision + recall)
print('VADER X OPLEXICON')
print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision, 2),'%',
      '\nRecall: ', round(100 * recall, 2),'%',
      '\nF1 Score: ', round(100 * f1_score, 2),'%')

print('% neutral: ', round(100 * (conf_vals[0] / len(df)), 2))

{0: 34756, 'TP': 15147, 'FP': 9262, 'TN': 7017, 'FN': 3626}
VADER X OPLEXICON
Accuracy:  63.23 % 
Precision:  62.05 % 
Recall:  80.69 % 
F1 Score:  70.15 %
% neutral:  49.79


In [144]:
rdf = pd.DataFrame(columns=['Sentiment Lexicon', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])
d = {
    'Sentiment Lexicon': 'OpLexicon',
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1_score
}
rdf = rdf.append(d, ignore_index=True)
rdf

,Sentiment Lexicon,Accuracy,Precision,Recall,F1-Score
0,OpLexicon,0.632318,0.62055,0.80685,0.701542


In [145]:
clean_df_columns = [
#     'text',
    'created_at',
    'id_user',
    'final_score',
    'final_polarity',
    'oplexicon_sentiment_analysis_score',
    'oplexicon_sentiment_analysis_polarity',
#     'sentistrength_sentiment_analysis_score',
#     'sentistrength_sentiment_analysis_polarity',
    'sentilexpt_sentiment_analysis_score',
    'sentilexpt_sentiment_analysis_polarity'
]

# VADER X SENTISTRENGTH

tweets = [data[d]['tweets'] for d in data]

final_tweets = []
for i in tweets:
    final_tweets.extend(i)
    
df = pd.DataFrame(final_tweets)

del df[0]
del df[1]
del df[4]
del df[5]
del df[6]
del df[18]
del df[19]
del df[20]

df.columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

clean_df(df)

df['vader_prediction'] = df['vader_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
df['sentistrength_prediction'] = df['sentistrength_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))

df['accuracy'] = df.apply(lambda x: 1 if x['vader_prediction'] == x['sentistrength_prediction'] else 0, axis=1)

df.head()

,text,vader_sentiment_analysis_score,vader_sentiment_analysis_polarity,sentistrength_sentiment_analysis_score,sentistrength_sentiment_analysis_polarity,vader_prediction,sentistrength_prediction,accuracy
0,"Atenção, estou mudando os moveis da sala de lu...",-0.4588,neg,0.00,neu,-1,0,0
1,RT @andreariell: o cancelamento dos participan...,0.0000,neu,0.00,neu,0,0,1
2,RT @AndrewBrobston: NOVA ANTECIPAÇÃO\nBoa tard...,0.7734,pos,0.00,neu,1,0,0
3,"@DouglasMarendaz Eu amo muito meu pai, mas fal...",-0.8720,neg,0.25,pos,-1,1,0
4,@lilicebk Eu cresci numa casa cheia de gente e...,-0.6808,neg,0.00,neu,-1,0,0


In [146]:
df['conf_matrix'] = df.apply(lambda x: conf_matrix(x, 'vader_prediction', 'sentistrength_prediction'), axis=1)

conf_vals = df['conf_matrix'].value_counts().to_dict()
print(conf_vals)

accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
precision = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])
recall = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
f1_score = 2*precision*recall / (precision + recall)
print('VADER X SENTISTRENGTH')
print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision, 2),'%',
      '\nRecall: ', round(100 * recall, 2),'%',
      '\nF1 Score: ', round(100 * f1_score, 2),'%')

{0: 38361, 'TP': 13813, 'TN': 10343, 'FP': 4430, 'FN': 2861}
VADER X SENTISTRENGTH
Accuracy:  76.81 % 
Precision:  75.72 % 
Recall:  82.84 % 
F1 Score:  79.12 %


In [147]:
d = {
    'Sentiment Lexicon': 'SentiStrength',
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1_score
}
rdf = rdf.append(d, ignore_index=True)
rdf

,Sentiment Lexicon,Accuracy,Precision,Recall,F1-Score
0,OpLexicon,0.632318,0.620550,0.806850,0.701542
1,SentiStrength,0.768150,0.757167,0.828415,0.791191


In [148]:
clean_df_columns = [
#     'text',
    'created_at',
    'id_user',
    'final_score',
    'final_polarity',
    'oplexicon_sentiment_analysis_score',
    'oplexicon_sentiment_analysis_polarity',
    'sentistrength_sentiment_analysis_score',
    'sentistrength_sentiment_analysis_polarity',
#     'sentilexpt_sentiment_analysis_score',
#     'sentilexpt_sentiment_analysis_polarity'
]

# VADER X SENTILEXPT

tweets = [data[d]['tweets'] for d in data]

final_tweets = []
for i in tweets:
    final_tweets.extend(i)
    
df = pd.DataFrame(final_tweets)

del df[0]
del df[1]
del df[4]
del df[5]
del df[6]
del df[18]
del df[19]
del df[20]

df.columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

clean_df(df)

df['vader_prediction'] = df['vader_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
df['sentilexpt_prediction'] = df['sentilexpt_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))

df['accuracy'] = df.apply(lambda x: 1 if x['vader_prediction'] == x['sentilexpt_prediction'] else 0, axis=1)

df.head()

,text,vader_sentiment_analysis_score,vader_sentiment_analysis_polarity,sentilexpt_sentiment_analysis_score,sentilexpt_sentiment_analysis_polarity,vader_prediction,sentilexpt_prediction,accuracy
0,"Atenção, estou mudando os moveis da sala de lu...",-0.4588,neg,0.00,neu,-1,0,0
1,RT @andreariell: o cancelamento dos participan...,0.0000,neu,0.00,neu,0,0,1
2,RT @AndrewBrobston: NOVA ANTECIPAÇÃO\nBoa tard...,0.7734,pos,0.00,neu,1,0,0
3,"@DouglasMarendaz Eu amo muito meu pai, mas fal...",-0.8720,neg,-0.25,neg,-1,-1,1
4,@lilicebk Eu cresci numa casa cheia de gente e...,-0.6808,neg,-0.25,neg,-1,-1,1


In [149]:
df['conf_matrix'] = df.apply(lambda x: conf_matrix(x, 'vader_prediction', 'sentilexpt_prediction'), axis=1)

conf_vals = df['conf_matrix'].value_counts().to_dict()
print(conf_vals)

accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
precision = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])
recall = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
f1_score = 2*precision*recall / (precision + recall)
print('VADER X SENTILEXPT')
print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision, 2),'%',
      '\nRecall: ', round(100 * recall, 2),'%',
      '\nF1 Score: ', round(100 * f1_score, 2),'%')

{0: 38089, 'TP': 12377, 'TN': 10127, 'FP': 5157, 'FN': 4058}
VADER X SENTILEXPT
Accuracy:  70.95 % 
Precision:  70.59 % 
Recall:  75.31 % 
F1 Score:  72.87 %


In [150]:
d = {
    'Sentiment Lexicon': 'Sentilex-PT',
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1_score
}
rdf = rdf.append(d, ignore_index=True)
rdf

,Sentiment Lexicon,Accuracy,Precision,Recall,F1-Score
0,OpLexicon,0.632318,0.620550,0.806850,0.701542
1,SentiStrength,0.768150,0.757167,0.828415,0.791191
2,Sentilex-PT,0.709480,0.705886,0.753088,0.728723


In [151]:
def create_data():
    bar_chart_data = []

    count = 1
    for i in data:
        if data[i]['bigfive']:
            tweets = create_tweets_pd(data, i)
#             columns_to_delete = ['id', 'id_str_twitter', "text", 'created_at', 'favorite_count', 'retweet_count', 'lang', "id_user",
#                              "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
#                             "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
#                             "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
#                             "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
#                             'text_updated', 'is_retweet', 'retweet_updated']

#             for c in columns_to_delete:
#                 del tweets[c]

            d = ['P{}'.format(count),
                 len(tweets.query("final_polarity == 'pos'")),
                 len(tweets.query("final_polarity == 'neu'")), 
                 len(tweets.query("final_polarity == 'neg'")),
                 (len(tweets.query("final_polarity == 'pos'")) / len(tweets)) * 100,
                 (len(tweets.query("final_polarity == 'neu'")) / len(tweets)) * 100,
                 (len(tweets.query("final_polarity == 'neg'")) / len(tweets)) * 100,
                 data[i]['bigfive'][0][1],
                 data[i]['bigfive'][0][2],
                 data[i]['bigfive'][0][3],
                 data[i]['bigfive'][0][4],
                 data[i]['bigfive'][0][5]
                ]
            bar_chart_data.append(d)
            count += 1

    df = pd.DataFrame(bar_chart_data, columns=[
        "Participant","pos", "neu", "neg", 
        "por_pos","por_neu","por_neg", "O", "C", "E", "A", "N"])
    
    biglist = ['O', 'C', 'E', 'A', 'N']
    for b in biglist:
        res = []
        for r in df[b]:
            if r <= 10:
                res.append(1)
            elif r > 10 and r < 21:
                res.append(2)
            elif r > 20 and r < 31:
                res.append(3)
            elif r > 30 and r < 41:
                res.append(4)
            elif r > 40:
                res.append(5)
        
        df['{}_Coded'.format(b)] = res
    
    return df

In [152]:
df = create_data()

In [153]:
# biglist = ['O', 'C', 'E', 'A', 'N']
# for b in biglist:
#     res = []
#     for r in df[b]:
#         if r <= 10:
#             res.append(1)
#         elif r > 10 and r < 21:
#             res.append(2)
#         elif r > 20 and r < 31:
#             res.append(3)
#         elif r > 30 and r < 41:
#             res.append(4)
#         elif r > 40:
#             res.append(5)
        
#     df['{}_Coded'.format(b)] = res
    
# df

In [154]:
pg.normality(df)

,W,pval,normal
pos,0.893909,0.092046,True
neu,0.925103,0.260186,True
neg,0.970725,0.886208,True
por_pos,0.965332,0.808860,True
por_neu,0.848046,0.020897,False
por_neg,0.952359,0.597902,True
O,0.947355,0.520445,True
C,0.874583,0.048688,False
E,0.953392,0.614493,True
A,0.907478,0.144836,True


In [155]:
pg.homoscedasticity(df)

,W,pval,equal_var
levene,0.463951,0.942406,True


In [156]:
pg.anova(dv='pos', between=['O'], data=df).round(3)

,Source,ddof1,ddof2,F,p-unc,np2
0,O,11,2,9.503,0.099,0.981


In [157]:
pg.anova(dv='pos', between=['O', 'E'], data=df).round(3)

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1069: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac1 = ms_fac1 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1070: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac2 = ms_fac2 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1071: RuntimeWarning: divide by zero encountered in double_scalars
  fval_inter = ms_inter / ms_resid


,Source,SS,DF,MS,F,np2
0,O,2.347195e+07,11,2133813.323,-inf,1.0
1,E,1.978227e+07,10,1978227.205,-inf,1.0
2,O * E,-1.933319e+07,110,-175756.285,inf,1.0
3,Residual,0.000000e+00,-118,-0.000,NaN,NaN


In [158]:
pg.anova(dv='pos', between=['O', 'A'], data=df).round(3)

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1069: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac1 = ms_fac1 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1070: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac2 = ms_fac2 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1071: RuntimeWarning: divide by zero encountered in double_scalars
  fval_inter = ms_inter / ms_resid


,Source,SS,DF,MS,F,np2
0,O,2.347195e+07,11,2133813.323,-inf,1.0
1,A,1.957433e+07,9,2174925.413,-inf,1.0
2,O * A,-1.912525e+07,99,-193184.324,inf,1.0
3,Residual,0.000000e+00,-106,-0.000,NaN,NaN


In [159]:
pg.anova(dv='pos', between=['O', 'N'], data=df).round(3)

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1069: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac1 = ms_fac1 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1070: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac2 = ms_fac2 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1071: RuntimeWarning: divide by zero encountered in double_scalars
  fval_inter = ms_inter / ms_resid


,Source,SS,DF,MS,F,np2
0,O,2.347195e+07,11,2133813.323,-inf,1.0
1,N,1.781982e+07,10,1781981.671,-inf,1.0
2,O * N,-1.737074e+07,110,-157915.782,inf,1.0
3,Residual,0.000000e+00,-118,-0.000,NaN,NaN


In [160]:
stats.f_oneway(df['pos'], df['O'])

F_onewayResult(statistic=39.23114837445394, pvalue=1.2521582756852636e-06)

In [161]:
model = ols('pos ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O,1.830187e+06,1.0,1.288527,0.289182
C,3.371972e+06,1.0,2.374006,0.161938
E,3.194705e+06,1.0,2.249203,0.172071
A,2.804223e+06,1.0,1.974287,0.197615
N,4.970656e+06,1.0,3.499544,0.098300
Residual,1.136298e+07,8.0,NaN,NaN


In [162]:
model = ols('neg ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O,2.144798e+05,1.0,0.555692,0.477329
C,1.426734e+06,1.0,3.696498,0.090747
E,8.201345e+04,1.0,0.212487,0.657098
A,1.464984e+06,1.0,3.795599,0.087230
N,1.292671e+06,1.0,3.349156,0.104626
Residual,3.087753e+06,8.0,NaN,NaN


In [163]:
model = ols('por_pos ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O,9.511502,1.0,0.072632,0.794363
C,44.295074,1.0,0.338248,0.576858
E,126.794055,1.0,0.968231,0.353941
A,43.444518,1.0,0.331753,0.580466
N,34.180853,1.0,0.261013,0.623222
Residual,1047.635134,8.0,NaN,NaN


In [164]:
model = ols('por_neg ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O,3.976233,1.0,0.073306,0.793437
C,27.900691,1.0,0.514380,0.493654
E,95.081296,1.0,1.752930,0.222087
A,24.246242,1.0,0.447006,0.522580
N,0.694367,1.0,0.012801,0.912705
Residual,433.930918,8.0,NaN,NaN


In [165]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['O'], alpha=0.05)
print(m_comp)

     Multiple Comparison of Means - Tukey HSD, FWER=0.05     
group1 group2  meandiff  p-adj     lower      upper    reject
-------------------------------------------------------------
    17     22     1326.0  0.722  -5665.1917  8317.1917  False
    17     29  -676.3333    0.9  -6384.6174  5031.9508  False
    17     31      240.0    0.9  -6751.1917  7231.1917  False
    17     32      353.0    0.9  -6638.1917  7344.1917  False
    17     35    -1126.0 0.8096  -8117.1917  5865.1917  False
    17     36    -1290.0 0.7377  -8281.1917  5701.1917  False
    17     37      -25.0    0.9  -7016.1917  6966.1917  False
    17     39     3559.0 0.1768  -3432.1917 10550.1917  False
    17     40     2268.0 0.3721  -4723.1917  9259.1917  False
    17     41       -7.0    0.9  -6998.1917  6984.1917  False
    17     43      758.0    0.9  -6233.1917  7749.1917  False
    22     29 -2002.3333 0.3299  -7710.6174  3705.9508  False
    22     31    -1086.0 0.8271  -8077.1917  5905.1917  False
    22  

In [166]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['E'], alpha=0.05)
print(m_comp)

     Multiple Comparison of Means - Tukey HSD, FWER=0.05     
group1 group2  meandiff  p-adj     lower      upper    reject
-------------------------------------------------------------
    19     25     -113.0    0.9 -11524.7604 11298.7604  False
    19     27    -1395.0    0.9 -12806.7604 10016.7604  False
    19     28     -494.0    0.9 -11905.7604 10917.7604  False
    19     31   -88.3333    0.9  -9405.9967    9229.33  False
    19     32    -1479.0    0.9 -12890.7604  9932.7604  False
    19     33     -353.0    0.9 -11764.7604 11058.7604  False
    19     34     -779.5    0.9 -10662.3744  9103.3744  False
    19     36     -378.0    0.9 -11789.7604 11033.7604  False
    19     38     1915.0    0.9  -9496.7604 13326.7604  False
    19     39     3206.0 0.7009  -8205.7604 14617.7604  False
    25     27    -1282.0    0.9 -12693.7604 10129.7604  False
    25     28     -381.0    0.9 -11792.7604 11030.7604  False
    25     31    24.6667    0.9  -9292.9967    9342.33  False
    25  

In [167]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['N'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2 meandiff p-adj    lower      upper   reject
---------------------------------------------------------
     8     15    616.0 0.7196 -1637.7716 2869.7716  False
     8     16  -1688.0 0.1538 -4290.4313  914.4313  False
     8     19  -1388.0 0.2415 -3990.4313 1214.4313  False
     8     21   -838.0 0.4823 -3091.7716 1415.7716  False
     8     23   -362.0    0.9 -2964.4313 2240.4313  False
     8     26    127.0    0.9 -2475.4313 2729.4313  False
     8     28   -296.0    0.9 -2549.7716 1957.7716  False
     8     29  -1237.0 0.3079 -3839.4313 1365.4313  False
     8     35   -955.0 0.4921 -3557.4313 1647.4313  False
     8     39  -1367.0 0.2497 -3969.4313 1235.4313  False
    15     16  -2304.0  0.047 -4557.7716  -50.2284   True
    15     19  -2004.0 0.0687 -4257.7716  249.7716  False
    15     21  -1454.0 0.0937 -3294.1968  386.1968  False
    15     23   -978.0 0.3685 -3231.7716 1275.7716  False
    15     26 

In [168]:
model = ols('pos ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O_Coded,8.902113e+04,1.0,0.039372,0.847664
C_Coded,3.440740e+05,1.0,0.152174,0.706647
E_Coded,8.012090e+05,1.0,0.354353,0.568112
A_Coded,3.066058e+06,1.0,1.356033,0.277775
N_Coded,3.260696e+05,1.0,0.144212,0.714012
Residual,1.808840e+07,8.0,NaN,NaN


In [169]:
model = ols('neg ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O_Coded,1.345767e+05,1.0,0.325984,0.583712
C_Coded,9.772632e+05,1.0,2.367215,0.162468
E_Coded,2.356198e+04,1.0,0.057074,0.817188
A_Coded,1.469353e+06,1.0,3.559198,0.095929
N_Coded,2.417214e+05,1.0,0.585519,0.466129
Residual,3.302660e+06,8.0,NaN,NaN


In [170]:
model = ols('por_pos ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O_Coded,17.621889,1.0,0.166045,0.694337
C_Coded,369.511703,1.0,3.481776,0.099021
E_Coded,53.749119,1.0,0.506459,0.496900
A_Coded,113.417491,1.0,1.068692,0.331477
N_Coded,37.994153,1.0,0.358005,0.566167
Residual,849.018776,8.0,NaN,NaN


In [171]:
model = ols('por_neg ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O_Coded,1.784965,1.0,0.036524,0.853198
C_Coded,114.886055,1.0,2.350809,0.163759
E_Coded,34.550497,1.0,0.706975,0.424866
A_Coded,19.953848,1.0,0.408297,0.540694
N_Coded,4.634763,1.0,0.094837,0.765981
Residual,390.966954,8.0,NaN,NaN


In [172]:
pg.ttest(df['pos'], df['O'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,6.263477,26,two-sided,0.000001,"[1525.57, 3016.01]",2.367372,9554.025,0.999975


In [173]:
pg.ttest(df['pos'], df['C'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,6.239092,26,two-sided,0.000001,"[1516.71, 3007.14]",2.358155,9036.325,0.999973


In [174]:
pg.ttest(df['pos'], df['E'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,6.267857,26,two-sided,0.000001,"[1527.14, 3017.57]",2.369027,9650.067,0.999976


In [175]:
pg.ttest(df['pos'], df['A'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,6.2852,26,two-sided,0.000001,"[1533.43, 3023.86]",2.375582,1.004e+04,0.999978


In [176]:
pg.ttest(df['pos'], df['N'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,6.290441,26,two-sided,0.000001,"[1535.35, 3025.79]",2.377563,1.016e+04,0.999978


In [177]:
print(pg.mwu(df['pos'], df['O']))
print(pg.mwu(df['neg'], df['O']))

     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0


In [180]:
print(pg.mwu(df['pos'], df['C']))
print(pg.mwu(df['neg'], df['C']))

     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0


In [181]:
print(pg.mwu(df['pos'], df['E']))
print(pg.mwu(df['neg'], df['E']))

     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0


In [182]:
print(pg.mwu(df['pos'], df['A']))
print(pg.mwu(df['neg'], df['A']))

     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0


In [183]:
print(pg.mwu(df['pos'], df['N']))
print(pg.mwu(df['neg'], df['N']))

     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  196.0  two-sided  0.000007 -1.0   1.0


In [184]:
print(pg.mwu(df['por_pos'], df['O']))
print(pg.mwu(df['por_neg'], df['O']))

     U-val       tail     p-val       RBC      CLES
MWU  180.0  two-sided  0.000179 -0.836735  0.918367
     U-val       tail     p-val       RBC      CLES
MWU  128.0  two-sided  0.175036 -0.306122  0.653061


In [185]:
print(pg.mwu(df['por_pos'], df['C']))
print(pg.mwu(df['por_neg'], df['C']))

     U-val       tail     p-val      RBC     CLES
MWU  142.0  two-sided  0.045342 -0.44898  0.72449
     U-val       tail    p-val       RBC      CLES
MWU   55.0  two-sided  0.05053  0.438776  0.280612


In [186]:
print(pg.mwu(df['por_pos'], df['E']))
print(pg.mwu(df['por_neg'], df['E']))

     U-val       tail     p-val       RBC      CLES
MWU  188.0  two-sided  0.000039 -0.918367  0.959184
     U-val       tail    p-val      RBC     CLES
MWU  142.0  two-sided  0.04549 -0.44898  0.72449


In [187]:
print(pg.mwu(df['por_pos'], df['A']))
print(pg.mwu(df['por_neg'], df['A']))

     U-val       tail     p-val       RBC      CLES
MWU  194.0  two-sided  0.000011 -0.979592  0.989796
     U-val       tail     p-val       RBC      CLES
MWU  177.0  two-sided  0.000303 -0.806122  0.903061


In [188]:
print(pg.mwu(df['por_pos'], df['N']))
print(pg.mwu(df['por_neg'], df['N']))

     U-val       tail     p-val       RBC      CLES
MWU  193.0  two-sided  0.000014 -0.969388  0.984694
     U-val       tail     p-val       RBC      CLES
MWU  175.0  two-sided  0.000437 -0.785714  0.892857


In [190]:
print(pg.kruskal(data=df, dv='pos', between='O'))
print(pg.kruskal(data=df, dv='pos', between='O_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='O'))
print(pg.kruskal(data=df, dv='por_pos', between='O_Coded'))
print(pg.kruskal(data=df, dv='neg', between='O'))
print(pg.kruskal(data=df, dv='neg', between='O_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='O'))
print(pg.kruskal(data=df, dv='por_neg', between='O_Coded'))

        Source  ddof1          H     p-unc
Kruskal      O     11  12.885714  0.300861
          Source  ddof1         H     p-unc
Kruskal  O_Coded      3  0.836735  0.840662
        Source  ddof1          H    p-unc
Kruskal      O     11  12.542857  0.32425
          Source  ddof1         H     p-unc
Kruskal  O_Coded      3  3.487755  0.322354
        Source  ddof1          H    p-unc
Kruskal      O     11  12.542857  0.32425
          Source  ddof1         H     p-unc
Kruskal  O_Coded      3  2.589796  0.459281
        Source  ddof1     H     p-unc
Kruskal      O     11  12.2  0.348799
          Source  ddof1         H     p-unc
Kruskal  O_Coded      3  4.418367  0.219688


In [191]:
print(pg.kruskal(data=df, dv='pos', between='C'))
print(pg.kruskal(data=df, dv='pos', between='C_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='C'))
print(pg.kruskal(data=df, dv='por_pos', between='C_Coded'))
print(pg.kruskal(data=df, dv='neg', between='C'))
print(pg.kruskal(data=df, dv='neg', between='C_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='C'))
print(pg.kruskal(data=df, dv='por_neg', between='C_Coded'))

        Source  ddof1         H     p-unc
Kruskal      C      7  11.52381  0.117347
          Source  ddof1     H     p-unc
Kruskal  C_Coded      1  0.02  0.887537
        Source  ddof1        H     p-unc
Kruskal      C      7  9.92381  0.192932
          Source  ddof1     H     p-unc
Kruskal  C_Coded      1  3.38  0.065992
        Source  ddof1    H     p-unc
Kruskal      C      7  9.0  0.252656
          Source  ddof1    H     p-unc
Kruskal  C_Coded      1  2.0  0.157299
        Source  ddof1         H     p-unc
Kruskal      C      7  7.447619  0.383806
          Source  ddof1     H     p-unc
Kruskal  C_Coded      1  3.92  0.047715


In [192]:
print(pg.kruskal(data=df, dv='pos', between='E'))
print(pg.kruskal(data=df, dv='pos', between='E_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='E'))
print(pg.kruskal(data=df, dv='por_pos', between='E_Coded'))
print(pg.kruskal(data=df, dv='neg', between='E'))
print(pg.kruskal(data=df, dv='neg', between='E_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='E'))
print(pg.kruskal(data=df, dv='por_neg', between='E_Coded'))

        Source  ddof1         H     p-unc
Kruskal      E     10  8.514286  0.578734
          Source  ddof1         H     p-unc
Kruskal  E_Coded      2  0.984762  0.611169
        Source  ddof1          H    p-unc
Kruskal      E     10  10.990476  0.35826
          Source  ddof1         H     p-unc
Kruskal  E_Coded      2  1.024762  0.599068
        Source  ddof1         H     p-unc
Kruskal      E     10  8.361905  0.593532
          Source  ddof1        H     p-unc
Kruskal  E_Coded      2  0.59619  0.742231
        Source  ddof1          H     p-unc
Kruskal      E     10  11.704762  0.305302
          Source  ddof1         H     p-unc
Kruskal  E_Coded      2  1.024762  0.599068


In [193]:
print(pg.kruskal(data=df, dv='pos', between='A'))
print(pg.kruskal(data=df, dv='pos', between='A_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='A'))
print(pg.kruskal(data=df, dv='por_pos', between='A_Coded'))
print(pg.kruskal(data=df, dv='neg', between='A'))
print(pg.kruskal(data=df, dv='neg', between='A_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='A'))
print(pg.kruskal(data=df, dv='por_neg', between='A_Coded'))

        Source  ddof1     H     p-unc
Kruskal      A      9  10.3  0.326749
          Source  ddof1         H     p-unc
Kruskal  A_Coded      2  2.790476  0.247774
        Source  ddof1         H     p-unc
Kruskal      A      9  7.857143  0.548605
          Source  ddof1         H     p-unc
Kruskal  A_Coded      2  0.768254  0.681045
        Source  ddof1          H     p-unc
Kruskal      A      9  11.785714  0.225661
          Source  ddof1         H     p-unc
Kruskal  A_Coded      2  3.739683  0.154148
        Source  ddof1         H     p-unc
Kruskal      A      9  9.171429  0.421603
          Source  ddof1         H     p-unc
Kruskal  A_Coded      2  0.873016  0.646289


In [194]:
print(pg.kruskal(data=df, dv='pos', between='N'))
print(pg.kruskal(data=df, dv='pos', between='N_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='N'))
print(pg.kruskal(data=df, dv='por_pos', between='N_Coded'))
print(pg.kruskal(data=df, dv='neg', between='N'))
print(pg.kruskal(data=df, dv='neg', between='N_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='N'))
print(pg.kruskal(data=df, dv='por_neg', between='N_Coded'))

        Source  ddof1          H     p-unc
Kruskal      N     10  11.571429  0.314762
          Source  ddof1         H     p-unc
Kruskal  N_Coded      3  1.689796  0.639205
        Source  ddof1          H     p-unc
Kruskal      N     10  11.371429  0.329322
          Source  ddof1         H     p-unc
Kruskal  N_Coded      3  0.193878  0.978572
        Source  ddof1          H     p-unc
Kruskal      N     10  12.742857  0.238406
          Source  ddof1         H     p-unc
Kruskal  N_Coded      3  1.779592  0.619386
        Source  ddof1          H     p-unc
Kruskal      N     10  11.371429  0.329322
          Source  ddof1         H     p-unc
Kruskal  N_Coded      3  0.242857  0.970391
